In [101]:
import scipy.io
MSdata = scipy.io.loadmat('C:/Users/lhren/Desktop/Fall 2017/4771/data/MSdata.mat')

In [128]:
from sklearn import linear_model
from sklearn.metrics import r2_score
import statsmodels.api as sm
import numpy as np
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
import random
import sys

In [178]:
trainX = MSdata['trainx']
testX = MSdata['testx']
trainY = MSdata['trainy'][:,0].astype(float)

In [179]:
#feature space transformation
train_xe = trainX[:,0:12]
train_xv = trainX[:,12:]
poly = PolynomialFeatures(degree=2)
eX = poly.fit_transform(train_xe)[:,1:]
#Train = list(trainX.T)
Train = list(eX.T)

for j in range(4):
    p = j+1
    
    Train += list((train_xv**p).T)
    

Train.append(list(trainY))
Train = np.array(Train).T
#random.shuffle(Train)

In [180]:
#poly = PolynomialFeatures(degree=5)
#pca = PCA(n_components=12)
#X = pca.fit_transform(trainX[:,0:12])
#newX = poly.fit_transform(X)[:,1:]
#random.shuffle(train)

#first pass regression
d = len(Train[0])-1
X = Train[:,0:d]
Y = Train[:,d]
regr = linear_model.LinearRegression(normalize=True)
regr.fit(X,Y)
pred_trainY = regr.predict(X)

In [181]:
r2_score(trainY,pred_trainY)

0.29155755347006318

In [182]:
sum(abs(trainY-pred_trainY))*1.0/len(trainY)

6.5264254255626684

In [183]:
#delete large outliers
Train_two = []
for i in range(len(trainY)):
    if abs(trainY[i]-pred_trainY[i])<30:
        Train_two.append(Train[i])


In [184]:
#second pass regression
Train_two = np.array(Train_two)
d = len(Train_two[0])-1
X2 = Train_two[:,:d]
Y2 = Train_two[:,d]
regr_2 = linear_model.LinearRegression(normalize=True)
regr_2.fit(X2,Y2)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [186]:
pred_trainY2 = regr_2.predict(X)
r2_score(trainY,pred_trainY2)

0.28189623371774397

In [187]:
sum(abs(trainY-pred_trainY2))*1.0/len(trainY)

6.4041259625051783

In [188]:
#write test prediction results
test_xe = testX[:,0:12]
test_xv = testX[:,12:]
poly = PolynomialFeatures(degree=2)
eX = poly.fit_transform(test_xe)[:,1:]
#Train = list(trainX.T)
Test = list(eX.T)
for j in range(4):
    p = j+1
    Test += list((test_xv**p).T)

Test = np.array(Test).T
pred_testY = regr_2.predict(Test)


In [189]:
f = open('output.csv','w')

In [190]:
i=1
f.write('dataid,prediction\n')
for y in pred_testY:
    f.write(str(i)+','+str(y)+'\n')
    i+=1
f.close()

In [37]:
#other stuff
count = np.zeros(10)
for y in trainY:
    era = int(y/10-192)
    count[era]+=1
print count
    
    

[  2.04000000e+02   2.39000000e+02   2.95000000e+02   2.82700000e+03
   1.05730000e+04   2.23490000e+04   3.76130000e+04   1.12133000e+05
   2.69118000e+05   8.36400000e+03]


In [ ]:
x = trainX[0][12:]
print x
cov = np.zeros((12,12))
i=12
while i>0:
    cov[12-i,(12-i):] = x[:i]
    cov[(12-i):,12-i] = x[:i]
    x = x[i:]
    i = i-1
#print cov
for i in range(12):
    for j in range(12):
        cov[i][j] = cov[i][j]/(np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
#print cov

    